In [1]:
import config
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
name_file = pd.read_csv('league_teams.csv')

In [3]:
hrefs = name_file['id']

In [44]:
url_pattern = 'index.php?p=public_team_info_players.php&team_id={}'

In [4]:
main_pattern = 'https://www.hockeyarena.net/en/'

In [6]:
def get_html(url):
    html = config.s.get(url, cookies=config.s.cookies)
    return html.text

In [7]:
def get_soup(html):
    soup = BeautifulSoup(html, 'lxml')
    return soup

In [49]:
total_players = {}

In [48]:
def get_table_data(soup):
    inside_dict = {}
    table = soup.find_all('table')
    team_name = ' '.join(table[0].text.strip().split()[3:])
    print(team_name)
    
    players = table[1].find_all('tr', {'class':['sr1','sr2']})
    for i in range(len(players)):
        player = players[i].text.strip().replace('\r','').replace('\t','').split('\n')
        info = {player[0]:player[1:] for p in player}
        inside_dict.update(info)
        
    clean_dict(inside_dict)
    clean_dict2(inside_dict)
    add_team_name(inside_dict,team_name)
    total_players.update(inside_dict)

In [23]:
def url_iterator(id):
    url = main_pattern + url_pattern.format(id)
    get_table_data(get_soup(get_html(url)))

In [24]:
def get_team_id(string):
    stri = re.findall(r"\d+", string)
    clean = str(stri)[2:-2]   #
    return clean

#### dict cleaners

In [25]:
def clean_dict(dic):
    for value in dic:
        if len(dic[value]) > 12:
            try:
                dic[value].remove('\xa0')
            except:
                ValueError

In [26]:
def clean_dict2(dic):
    for value in dic:
        if len(dic[value]) > 11:
            try:
                dic[value].remove('')
            except:
                ValueError

In [47]:
def add_team_name(dic,team_name):
    for value in dic.items():
        value[1].insert(0,str(team_name))

### launch the stuff

In [50]:
for i in hrefs:
    url_iterator(get_team_id(i))

Soda Beņķa Sildītāji
Arch Enemy
HC Vilkatis
JOKISS
Ergli Elite: Revolution
HC Jelgava Stars
Patria
HC Skonto
Traveling Wilburys
LATGALE
Rockets
Kadri
Refleks
The BDM team
hk DEMONS
Löfbergs Lila


In [51]:
len(total_players)

520

### columns

In [56]:
cols = table[1].findAll('td', class_='ydt')

In [57]:
clean_cols = []

In [58]:
for col in cols:
    col = col.text
    clean_cols.append(col)

In [59]:
clean_cols

['\xa0Name',
 'Age',
 'AI',
 'GP',
 'Sho',
 'Goa',
 'Ass',
 '+/-',
 'PIM',
 'SA',
 'GA',
 'SV%']

In [75]:
df = pd.DataFrame.from_dict(total_players, columns=clean_cols, orient='index')

In [76]:
df.columns

Index([' Name', 'Age', 'AI', 'GP', 'Sho', 'Goa', 'Ass', '+/-', 'PIM', 'SA',
       'GA', 'SV%'],
      dtype='object')

### strip column names
#### reindex and sort by player names and GP

In [77]:
df = df.rename(columns=lambda x: x.strip())

In [80]:
df = df.sort_values(by=['Name', 'GP'],ascending=[True, False]).reset_index()

In [81]:
df.to_csv('league_teams_players.csv')

In [83]:
df[:13]

,index,Name,Age,AI,GP,Sho,Goa,Ass,+/-,PIM,SA,GA,SV%
0,Juraj Olša,Arch Enemy,33,400,9,12,0,4,3,8,0,0,0
1,Kimmo Ahomaa,Arch Enemy,32,474,9,16,1,5,11,2,0,0,0
2,Antyp Khrol,Arch Enemy,32,392,8,19,1,1,10,12,0,0,0
3,Sviatamir Salaujou,Arch Enemy,33,426,3,13,0,4,7,4,0,0,0
4,Tymoteusz Czubacki,Arch Enemy,31,403,17,36,0,6,30,14,0,0,0
5,Mattia Mantegazza,Arch Enemy,33,378,17,32,6,4,10,2,0,0,0
6,Bodo Herrmann,Arch Enemy,34,407,17,23,1,3,18,4,0,0,0
7,Bruce De Knegt,Arch Enemy,31,429,17,82,15,10,18,36,0,0,0
8,Barys Machayla,Arch Enemy,31,398,17,19,2,4,10,12,0,0,0
9,Miloslav Beneš,Arch Enemy,34,411,17,60,10,4,18,10,0,0,0
